In [50]:
from pymongo import MongoClient
import pandas as pd
import json as js

In [51]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [52]:
test = (db["cleanOffices.json"].aggregate([
   {
     '$geoNear': {
        'near': { 'type': "Point", 'coordinates': [ -73.996876 , 40.724604 ] },
        'distanceField': "dist.calculated",
        'maxDistance': 100,
        'query': { 'city': "New York" },
        'spherical': 'true'
     }
   }
]))

a = list(test)
a

[{'_id': ObjectId('5e4003ab35e23c13dc2b242c'),
  'name': 'AddThis',
  'category_code': 'advertising',
  'description': 'New York Office',
  'address1': '568 Broadway',
  'address2': '11th Floor, Suite 1105',
  'zip_code': '10012',
  'city': 'New York',
  'state_code': 'NY',
  'country_code': 'USA',
  'location': {'type': 'Point', 'coordinates': [-73.996876, 40.724604]},
  'dist': {'calculated': 0.0}},
 {'_id': ObjectId('5e4003ab35e23c13dc2b24d2'),
  'name': 'appssavvy',
  'category_code': 'advertising',
  'description': 'Headquarters',
  'address1': '594 Broadway',
  'address2': 'Suite 207',
  'zip_code': '10012',
  'city': 'New York',
  'state_code': 'NY',
  'country_code': 'USA',
  'location': {'type': 'Point', 'coordinates': [-73.996876, 40.724604]},
  'dist': {'calculated': 0.0}},
 {'_id': ObjectId('5e4003ab35e23c13dc2b24f7'),
  'name': 'AppNexus',
  'category_code': 'advertising',
  'description': 'Headquarters',
  'address1': '28 W. 23rd Street',
  'address2': 'Fifth Floor',
  'z

In [54]:
# Filter offices to New York. Just testing, my friend!
offices = pd.DataFrame(list(db['cleanOffices.json'].find({'city': 'New York'})))
offices.head()

,_id,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,5e4003ab35e23c13dc2b242c,AddThis,advertising,New York Office,568 Broadway,"11th Floor, Suite 1105",10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.996876, ..."
1,5e4003ab35e23c13dc2b242e,Joost,games_video,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,"{'type': 'Point', 'coordinates': [-74.0094471,..."
2,5e4003ab35e23c13dc2b242f,Jingle Networks,mobile,,475 Park Ave South,10th Floor,10016,New York,NY,USA,"{'type': 'Point', 'coordinates': [-122.173887,..."
3,5e4003ab35e23c13dc2b2432,Livestream,games_video,Livestream HQ,"111 8th Avenue, #1509",,10011,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.995625, ..."
4,5e4003ab35e23c13dc2b2434,Pando Networks,games_video,None,520 Broadway,10th Floor,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.99873, 4..."


In [64]:
# Function to know distance from office to a starbucks
def starbucks(coordinates):
    for index,row in offices.iterrows():
        coordinates = row['location']['coordinates']
    coffee = (db["starbucks"].aggregate([{'$geoNear': {'near': { 'type': "Point", 'coordinates': coordinates },
        'distanceField': "dist.calculated",
        'maxDistance': 10000,
        'query': { 'city': "New York" },
        'spherical': 'true'}}]))
    your_coffee = list(coffee)
    return your_coffee

#¿Puedo generalizar para cambiar el nombre de starbucks de db[starbucks] a airports? El problema viene cuando aplico apply 

def near(location):
    for index,row in offices.iterrows():
        coordinates = row['location']['coordinates']
    close = (db[f'{location}'].aggregate([{'$geoNear': {'near': { 'type': "Point", 'coordinates': coordinates },
        'distanceField': "dist.calculated",
        'maxDistance': 10000,
        'query': { 'city': "New York" },
        'spherical': 'true'}}]))
    closeto = list(coffee)
    return closeto

In [71]:
#offices['starbucks'] = offices['location'].apply(starbucks)
#offices['starbucks'] = offices['location'].apply(lambda x: near('starbucks'), axis=1) #Devuelve error en lambda

In [72]:
offices['airports'] = offices['location'].apply(lambda x: requestMap(lat, long, tipo='airport'), axis=1)

"""
# Etiquetas
airport
train_station
bar"""


# Google Maps requests
# Error requesting location
def requestMap(lat, long, tipo):
    token = os.getenv('MAPS_KEY')
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    for index,row in offices.iterrows():
        coordinates = row['location']['coordinates']
    params = {'location': coordinates,
              'type': tipo,
              'key' : token 
    }
    res = requests.get(url,params=params)
    if res != 200:
        raise ValueError('Bad Response')
    return res.json()

TypeError: <lambda>() got an unexpected keyword argument 'axis'